##### License
Licensed under the BSD 3-Clause License (the "License");

BSD 3-Clause License

Copyright (c) 2017, Pytorch contributors
All rights reserved.

Redistribution and use in source and binary forms, with or without
modification, are permitted provided that the following conditions are met:

* Redistributions of source code must retain the above copyright notice, this
  list of conditions and the following disclaimer.

* Redistributions in binary form must reproduce the above copyright notice,
  this list of conditions and the following disclaimer in the documentation
  and/or other materials provided with the distribution.

* Neither the name of the copyright holder nor the names of its
  contributors may be used to endorse or promote products derived from
  this software without specific prior written permission.

THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS"
AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE
IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE
DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE
FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL
DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR
SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER
CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY,
OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE
OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.

In the paper ['Fast Sparse Regression and Classification' (2008)](http://statweb.stanford.edu/~jhf/ftp/GPSpaper.pdf), [Jerome Friedman](https://statweb.stanford.edu/~jhf/) introduces the generalized path seeking (GPS) algorithm to directly construct, sequentially, a path in parameter space that approximates that for a given penalty $P(a)$ on the coffefficents $a$ of an associated regression model. This Jupyter notebook is an implementation of GPS that takes advantage of the 'autograd' facility provided by many machine learning frameworks for calculating gradients. [PyTorch (1.0.1)](https://pytorch.org/) will be used in this case.

In [2]:
import torch
from torch.autograd import Variable
import math
import numpy as np
import pandas as pd

Define a simple model and loss function to demonstrate how useful autograd will be for implementing GPS. $a$ is a vector of coefficients of the regression model and is exposed at the toplevel.

In [3]:
N = 9
a = torch.zeros(N,requires_grad=True, dtype=torch.float64)

def Fmodel(a, x):
    """ a is the coefficient vector of a linear regression model
        a[0] is the constant term
        x is a matrix of data where each row is an observation """
    return (x @ a[1:]) + a[0]

def Mse(t1,t2):
    """ mean square error """
    diff = t1-t2
    return torch.sum(diff*diff)/diff.numel()

For a concrete example, we will use data from [Stamey et al (1989)](https://www.ncbi.nlm.nih.gov/pubmed/2468795) that can be accessed online at [Robert Tibshirani's](https://statweb.stanford.edu/~tibs) website.

In [4]:
pdata = pd.read_csv("http://statweb.stanford.edu/~tibs/ElemStatLearn/datasets/prostate.data",
                    index_col=0,sep='\t')
# lpsa values are the y[i]'s that we want to predict
lpsa = torch.from_numpy(pdata.lpsa.values)
print(pdata.columns)
xvals = torch.from_numpy(pdata[['lcavol','lweight','age','lbph','svi','lcp','gleason','pgg45']].values)
print(xvals[:10])

Index(['lcavol', 'lweight', 'age', 'lbph', 'svi', 'lcp', 'gleason', 'pgg45',
       'lpsa', 'train'],
      dtype='object')
tensor([[-0.5798,  2.7695, 50.0000, -1.3863,  0.0000, -1.3863,  6.0000,  0.0000],
        [-0.9943,  3.3196, 58.0000, -1.3863,  0.0000, -1.3863,  6.0000,  0.0000],
        [-0.5108,  2.6912, 74.0000, -1.3863,  0.0000, -1.3863,  7.0000, 20.0000],
        [-1.2040,  3.2828, 58.0000, -1.3863,  0.0000, -1.3863,  6.0000,  0.0000],
        [ 0.7514,  3.4324, 62.0000, -1.3863,  0.0000, -1.3863,  6.0000,  0.0000],
        [-1.0498,  3.2288, 50.0000, -1.3863,  0.0000, -1.3863,  6.0000,  0.0000],
        [ 0.7372,  3.4735, 64.0000,  0.6152,  0.0000, -1.3863,  6.0000,  0.0000],
        [ 0.6931,  3.5395, 58.0000,  1.5369,  0.0000, -1.3863,  6.0000,  0.0000],
        [-0.7765,  3.5395, 47.0000, -1.3863,  0.0000, -1.3863,  6.0000,  0.0000],
        [ 0.2231,  3.2445, 63.0000, -1.3863,  0.0000, -1.3863,  6.0000,  0.0000]],
       dtype=torch.float64)


In [5]:
preds = Fmodel(a, xvals)

In [6]:
preds

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.], dtype=torch.float64, grad_fn=<AddBackward0>)

In [7]:
lpsa[:10]

tensor([-0.4308, -0.1625, -0.1625, -0.1625,  0.3716,  0.7655,  0.7655,  0.8544,
         1.0473,  1.0473], dtype=torch.float64)

In [8]:
loss = Mse(preds,lpsa)
print(loss)

tensor(7.4611, dtype=torch.float64, grad_fn=<DivBackward0>)


PyTorch keeps track of the mathematical operations performed on $a$ and will automatically calculate the gradient using reverse-mode differentiation that is often used for back-propagation in neural nets. In PyTorch, invoking the backword() method on a scalar tensor will automatically calculate the gradient. Note that gradients cumulate and will need to be zeroed out where appropriate for the calculation at hand.

In [9]:
loss.backward()
print(a)
print(a.grad)

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=torch.float64,
       requires_grad=True)
tensor([  -4.9568,   -8.6696,  -18.4120, -319.4542,   -1.0935,   -1.6087,
          -0.8643,  -34.0798, -148.0683], dtype=torch.float64)


One approach to updating the coefficients and minimize the loss is to iteratively move along the direction of the gradient. We will use this idea later to address a minor issue with the GPS algorithm as stated in the paper.

In [10]:
for i in range(10000):
    preds = Fmodel(a, xvals)
    loss = Mse(preds, lpsa)
    loss.backward()
    # no_grad() context because we do not want to calcualte but update a
    with torch.no_grad():
        a -= a.grad * 1e-5
        a.grad.zero_()
a_ref = a

In [11]:
print(loss)

tensor(0.8563, dtype=torch.float64, grad_fn=<DivBackward0>)


In [12]:
print(preds)

tensor([1.4125, 1.5940, 2.2870, 1.5660, 1.9234, 1.3732, 2.0223, 1.8837, 1.3424,
        1.8751, 1.9471, 1.7544, 2.4407, 2.2096, 2.0001, 2.1117, 2.4514, 2.3558,
        1.2002, 2.1494, 1.8940, 2.5385, 1.6929, 2.9630, 2.1405, 2.1942, 2.7896,
        2.2424, 3.0851, 2.4215, 2.0914, 2.0184, 2.2699, 1.6132, 1.8863, 2.2407,
        2.7582, 2.0804, 3.0699, 1.9769, 2.7702, 2.3134, 2.0606, 2.2945, 2.5000,
        2.2440, 4.1743, 2.7475, 1.5530, 2.3040, 2.7018, 2.2052, 2.8866, 3.0069,
        2.2331, 2.3985, 1.7394, 1.6258, 2.4212, 2.4656, 2.3084, 2.9030, 3.7387,
        3.2277, 2.1504, 2.3739, 3.2891, 2.6829, 1.9986, 2.5101, 2.9001, 2.7291,
        2.5677, 3.2882, 2.9128, 3.2606, 3.2614, 2.8288, 3.4287, 2.9003, 2.7113,
        2.9801, 3.1047, 3.3660, 2.5567, 3.2542, 2.0618, 2.5395, 3.2797, 3.4611,
        2.4237, 2.4141, 3.2438, 2.5904, 2.3703, 3.5405, 3.0644],
       dtype=torch.float64, grad_fn=<AddBackward0>)


In [13]:
print(lpsa)

tensor([-0.4308, -0.1625, -0.1625, -0.1625,  0.3716,  0.7655,  0.7655,  0.8544,
         1.0473,  1.0473,  1.2669,  1.2669,  1.2669,  1.3481,  1.3987,  1.4469,
         1.4702,  1.4929,  1.5581,  1.5994,  1.6390,  1.6582,  1.6956,  1.7138,
         1.7317,  1.7664,  1.8001,  1.8165,  1.8485,  1.8946,  1.9242,  2.0082,
         2.0082,  2.0215,  2.0477,  2.0857,  2.1576,  2.1917,  2.2138,  2.2773,
         2.2976,  2.3076,  2.3273,  2.3749,  2.5217,  2.5533,  2.5688,  2.5688,
         2.5915,  2.5915,  2.6568,  2.6776,  2.6844,  2.6912,  2.7047,  2.7180,
         2.7881,  2.7942,  2.8064,  2.8124,  2.8420,  2.8536,  2.8536,  2.8820,
         2.8820,  2.8876,  2.9205,  2.9627,  2.9627,  2.9730,  3.0131,  3.0374,
         3.0564,  3.0750,  3.2753,  3.3375,  3.3928,  3.4356,  3.4579,  3.5130,
         3.5160,  3.5308,  3.5653,  3.5709,  3.5877,  3.6310,  3.6801,  3.7124,
         3.9843,  3.9936,  4.0298,  4.1296,  4.3851,  4.6844,  5.1431,  5.4775,
         5.5829], dtype=torch.float64)


Central to the implementation of GPS are equations (24), (25), and (26) defined on page 6 of the article - reproduced below. Equations (24) and (25) captures the gradient of empirical 'risk' $\hat{R}(a)$ and the penalty $P(a)$ used to regularized the model, respectively. Both are directional vectors in the parameter space of a regression problem. While (24) is directly depedent on the model and data, (25) has a more 'universal' nature in that they are gradients of a penalty with respect to parameters -- which typically have a form applicable across models. Note that $\nu$ is a parameterization of the steps size in parameter space, and the 'hat' symbol e.g. ($\hat{R}$ and $\hat{a}$) signifies empirical quantities that are explicit dependent on the observed data.

$$
\begin{eqnarray}
g_{j}(\nu) & = & - & 
\left[\frac{\partial \hat{R}(a)}{\partial a_{j}}\right]_{a=\hat{a}(\nu)} & \hspace{1in} (24) \\
p_{j}(\nu) & = & & 
\left[\frac{\partial P(a)}{\partial \left| a_{j} \right|} \right]_{a=\hat{a}(\nu)} & \hspace{1in} (25) \\
\lambda_{j}(\nu) & = & & 
\frac{g_{j}(\nu)}{p_{j}(\nu)} & \hspace{1in} (26)
\end{eqnarray}
$$

Before putting together the whole algorithm, each piece will be demonstrated separately. Without going into details at this point (see Section 2.3 of the paper), $a$ will be reset to zero and slightly pushed along the negative gradient as a starting point for this demonstration. The difficulty with zero is not surprising when dealing with gradients of absolute values evaluate at zero.

In [14]:
N = 9
a = torch.zeros(N,requires_grad=True, dtype=torch.float64)

In [15]:
loss = Mse(Fmodel(a, xvals), lpsa)
loss.backward()
with torch.no_grad():
    a -= a.grad * 1e-5
    a.grad.zero_()

$$ Compute \{\lambda_{j}(\nu)\}^{n}_{1} $$

In [16]:
print(a)
R = Mse(Fmodel(a, xvals), lpsa);R.backward(); g = -a.grad; a.grad.zero_() #(24)
print(R)
#P = abs(a).pow(1/2).sum(); P.backward(); p = abs(a.grad); a.grad.zero_() #(25)
P = abs(a).pow(2).sum(); P.backward(); p = abs(a.grad); a.grad.zero_() #(25)
print(P)
l = g/p #(26)
print(p)
print(g)

tensor([4.9568e-05, 8.6696e-05, 1.8412e-04, 3.1945e-03, 1.0935e-05, 1.6087e-05,
        8.6427e-06, 3.4080e-04, 1.4807e-03], dtype=torch.float64,
       requires_grad=True)
tensor(6.2674, dtype=torch.float64, grad_fn=<DivBackward0>)
tensor(1.2558e-05, dtype=torch.float64, grad_fn=<SumBackward0>)
tensor([9.9135e-05, 1.7339e-04, 3.6824e-04, 6.3891e-03, 2.1870e-05, 3.2174e-05,
        1.7285e-05, 6.8160e-04, 2.9614e-03], dtype=torch.float64)
tensor([  4.4702,   7.9575,  16.6355, 287.8595,   1.0111,   1.4853,   0.8695,
         30.7401, 133.4941], dtype=torch.float64)


$$ S = \{ j \, | \, \lambda_{j}(\nu) * \hat{a}_{j}(\nu) < 0 \} $$

In [17]:
# use element wise multiplication and less than 0 predicate
# to find elements with corresponding opposite sign
S = l*a < 0
print(l)
print(a)
print (S)

tensor([45092.3164, 45893.0302, 45175.6898, 45054.8953, 46234.0950, 46164.1531,
        50303.5029, 45100.0898, 45078.5758], dtype=torch.float64)
tensor([4.9568e-05, 8.6696e-05, 1.8412e-04, 3.1945e-03, 1.0935e-05, 1.6087e-05,
        8.6427e-06, 3.4080e-04, 1.4807e-03], dtype=torch.float64,
       requires_grad=True)
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=torch.uint8)


$$
\begin{eqnarray}
if \; (S = empty) \hspace{5pt} & j^{*} & = arg\,max_{j} & | \lambda_{j}(\nu) | \\
else \hspace{5pt} & j^{*} & = arg\,max_{j \in S} & | \lambda_{j}(\nu) | 
\end{eqnarray}
$$

In [18]:
# need to maintain idx location, i.e. need to keep tensor shape the same throughout
# torch.max() finds max element and respective index
if S.sum() > 0: # check for elements in set
    # non empty case (order different than stated algorithm)
    # S.double() for matching element types needed by PyTorch
    # element wise mult to zero out elements not meeting predicate condition
    # done this way to make sure idx refer to corresponding element
    val,idx = torch.max(S.double() * l.abs(), 0)
else:
    # empty case
    val,idx = torch.max(l.abs(), 0)
print(val,idx)

tensor(50303.5029, dtype=torch.float64) tensor(6)


$$ \hat{a}_{j^{*}}(\nu + \Delta \nu) = \hat{a}_{j^{*}}(\nu) + \Delta \nu * sign(\lambda_{j^{*}}(\nu)) \\
 \{ \hat{a}_{j}(\nu + \Delta \nu) = \hat{a}_{j}(\nu) \}_{j \ne j^{*}} $$
$$ \nu \leftarrow \nu + \Delta \nu $$

In [19]:
# update single component, a[idx], with new value
# here are the values at play for this iteration
print(idx, a[idx], l[idx])

tensor(6) tensor(8.6427e-06, dtype=torch.float64, grad_fn=<SelectBackward>) tensor(50303.5029, dtype=torch.float64)


Only a[idx] compoment is updated with a new value, the rest remain unchanged. The code would then be something along the line of:
```python
with torch.no_grad():
    a[idx] += del_nu * torch.sign(l[idx])
    a.grad.zero_()
```
Choosing $\Delta \nu$ is an implementation decision.
Note that $\Delta \nu$ is an implied change in the parameterized path of $a(\nu)$ that would bring about a change of $\Delta a$. Since $sign(\lambda_{j^{*}}(\nu))$ contributes only the sign of the change, $\Delta \nu$ is effectively the magnitude of $\Delta a$. 

Section 9.4 of the paper suggests one approach to setting the step size: chose $\Delta \nu$ to reduce the empirical risk $\hat{R}(\hat{a})$ by a fixed fraction $\epsilon$.

$$ \frac{\left [ \hat{R}(\hat{a}(\nu)) - \hat{R}(\hat{a}(\nu + \Delta \nu)) \right]}
{\hat{R}(\hat{a}(\nu))} = \epsilon $$

The algorithm updates one component $a_{j^{*}}$ at a time. An approximation for $\epsilon$ is then

$$ \left | \frac{g_{j^{*}}(\nu) * \Delta a_{j^{*}}}{\hat{R}(a(\nu))_{a=\hat{a}(\nu)}} \right | \approx \epsilon $$

With a choice of $\epsilon$ = 0.01,

In [20]:
with torch.no_grad():
    del_nu = 0.01 * (R / g[idx]).abs()
    a[idx] += del_nu * torch.sign(l[idx])
    R_post = Mse(Fmodel(a, xvals), lpsa)
    a.grad.zero_()
    
# should be 'close' to 0.01
print(1-(R_post/R))

tensor(0.0084, dtype=torch.float64, grad_fn=<RsubBackward1>)


The GPS algorithm with all the pieces composed together.

Line numbering are diffent from the paper because the breakdown above was composed as units that better match this description.
$$
\begin{array}{ll}
1 & Initialize: \nu = 0; \{\hat{a}_{j}(0) = 0\}_{1}^{n} \\
2 & Loop \; \{ \\
3 & \hspace{10pt} Compute \{\lambda_{j}(\nu)\}^{n}_{1} \\
4 & \hspace{10pt} S = \{ j \, | \, \lambda_{j}(\nu) * \hat{a}_{j}(\nu) < 0 \} \\
5 & \hspace{10pt} \begin{eqnarray}
if \; (S = empty) \hspace{5pt} & j^{*} & = arg\,max_{j} & | \lambda_{j}(\nu) | \\
else \hspace{10pt} & j^{*} & = arg\,max_{j \in S} & | \lambda_{j}(\nu) |
\end{eqnarray} \\
6 & \hspace{10pt} \hat{a}_{j^{*}}(\nu + \Delta \nu) = \hat{a}_{j^{*}}(\nu) + \Delta \nu * sign(\lambda_{j^{*}}(\nu)); \{ \hat{a}_{j}(\nu + \Delta \nu) = \hat{a}_{j}(\nu) \}_{j \ne j^{*}} \\
7 & \hspace{10pt} \nu \leftarrow \nu + \Delta \nu \\
8 & \} \; Until \; \lambda(\nu) = 0
\end{array}
$$

In [21]:
## Line 1
N = 9
a = torch.zeros(N,requires_grad=True, dtype=torch.float64)
loss = Mse(Fmodel(a, xvals), lpsa)
loss.backward()
with torch.no_grad():
    a -= a.grad * 1e-5
    a.grad.zero_()
nu = 0
ncount = 0
NMAX = 10000

## Line 2
while True:
    ## Line 3
    R = Mse(Fmodel(a, xvals), lpsa);R.backward(); g = -a.grad; a.grad.zero_() #(24)
    P = abs(a).pow(1).sum(); P.backward(); p = abs(a.grad); a.grad.zero_() #(25)
    l = g/p #(26)
    ## Line 4
    # use element wise multiplication and less than 0 predicate
    # to find elements with corresponding opposite sign
    S = l*a < 0
    ## Line 5
    # need to maintain idx location, i.e. need to keep tensor shape the same throughout
    # torch.max() finds max element and respective index
    if S.sum() > 0: # check for elements in set
        # non empty case (order different than stated algorithm)
        # S.double() for matching element types needed by PyTorch
        # element wise mult to zero out elements not meeting predicate condition
        # done this way to make sure idx refer to corresponding element
        val,idx = torch.max(S.double() * l.abs(), 0)
    else:
        # empty case
        val,idx = torch.max(l.abs(), 0)
    ## Line 6
    with torch.no_grad():
        # recall that no gradients should be calculated here while a is being updated
        del_nu = 0.01 * (R / g[idx]).abs()
        a[idx] += del_nu * torch.sign(l[idx])
        # should be 'close' to 0.01
        # R_post = mse(Fmodel(a, xvals), lpsa); print(1-(R_post/R))
        a.grad.zero_()
    ## Line 7
    nu += del_nu
    ## Line 8
    #print(l.sum())
    if (ncount % 100 == 0): 
        print(ncount,R.data,P.data)
    if (ncount > NMAX) or (l.abs().sum() <= 0):
        break
    ncount += 1

0 tensor(6.2674, dtype=torch.float64) tensor(0.0054, dtype=torch.float64)
100 tensor(2.3031, dtype=torch.float64) tensor(0.0243, dtype=torch.float64)
200 tensor(1.5662, dtype=torch.float64) tensor(0.0378, dtype=torch.float64)
300 tensor(1.0992, dtype=torch.float64) tensor(0.0550, dtype=torch.float64)
400 tensor(3.1681, dtype=torch.float64) tensor(0.0332, dtype=torch.float64)
500 tensor(1.1694, dtype=torch.float64) tensor(0.0514, dtype=torch.float64)
600 tensor(1.7747, dtype=torch.float64) tensor(0.0500, dtype=torch.float64)
700 tensor(4.8778, dtype=torch.float64) tensor(0.0326, dtype=torch.float64)
800 tensor(1.7930, dtype=torch.float64) tensor(0.0498, dtype=torch.float64)
900 tensor(1.1154, dtype=torch.float64) tensor(0.0672, dtype=torch.float64)
1000 tensor(6.0591, dtype=torch.float64) tensor(0.0403, dtype=torch.float64)
1100 tensor(2.2260, dtype=torch.float64) tensor(0.0536, dtype=torch.float64)
1200 tensor(3.6075, dtype=torch.float64) tensor(0.0528, dtype=torch.float64)
1300 tensor

In [22]:
a

tensor([4.9568e-05, 1.6942e-01, 1.8412e-04, 3.1989e-02, 1.0935e-05, 1.6087e-05,
        8.6427e-06, 3.4080e-04, 1.2265e-02], dtype=torch.float64,
       requires_grad=True)

In [23]:
a_ref

tensor([0.0041, 0.1258, 0.0433, 0.0262, 0.0232, 0.0315, 0.0709, 0.0301, 0.0105],
       dtype=torch.float64, requires_grad=True)

In [24]:
l

tensor([ -0.2527,   0.6937,  -0.6433, -18.8883,   0.2337,   0.2396,   0.7350,
         -1.6861,  -6.8309], dtype=torch.float64)

Define some families of $P(a)$ from the paper and their gradient. Here k will be a proxy for $\gamma$ (Power family) or $\beta$ (Generalized elastic net and Subset selection), and w are weights that serve the role of the parameters $a$.

In [31]:
def Power(w,k):
    return w.abs().pow(k)

def Power_grad(w,k):
    return (k*w.abs()*w.abs().pow(k-1))/w

def ENet(w,k):
    return ((k-1)*(w.pow(2)))/2+((2-k)*w.abs())

def ENet_grad(w,k):
    return ((2-k)*w.abs()+(k-1)*w.pow(2))/w

def SubsetSel(w,k):
    return torch.log((1-k)*w.abs()+k)

def SubsetSel_grad(w,k):
    return ((k-1)*w.abs())/((k-1)*w*w.abs()-k*w)

Probably a good idea to check PyTorch's autograd functionality againt those defined above.

In [32]:
print(a); a.grad.zero_()
tr = SubsetSel(a,1.5).sum()
tr.backward()
print(a.grad)
a.grad.zero_()
print(SubsetSel_grad(a,1.5))

tensor([4.9568e-05, 1.6942e-01, 1.8412e-04, 3.1989e-02, 1.0935e-05, 1.6087e-05,
        8.6427e-06, 3.4080e-04, 1.2265e-02], dtype=torch.float64,
       requires_grad=True)
tensor([-0.3333, -0.3533, -0.3334, -0.3369, -0.3333, -0.3333, -0.3333, -0.3334,
        -0.3347], dtype=torch.float64)
tensor([-0.3333, -0.3533, -0.3334, -0.3369, -0.3333, -0.3333, -0.3333, -0.3334,
        -0.3347], dtype=torch.float64, grad_fn=<DivBackward0>)


In [33]:
print(a);a.grad.zero_()
tr = Power(a,2).sum()
tr.backward()
print(a.grad)
a.grad.zero_()
print(Power_grad(a,2))

tensor([4.9568e-05, 1.6942e-01, 1.8412e-04, 3.1989e-02, 1.0935e-05, 1.6087e-05,
        8.6427e-06, 3.4080e-04, 1.2265e-02], dtype=torch.float64,
       requires_grad=True)
tensor([9.9135e-05, 3.3884e-01, 3.6824e-04, 6.3979e-02, 2.1870e-05, 3.2174e-05,
        1.7285e-05, 6.8160e-04, 2.4531e-02], dtype=torch.float64)
tensor([9.9135e-05, 3.3884e-01, 3.6824e-04, 6.3979e-02, 2.1870e-05, 3.2174e-05,
        1.7285e-05, 6.8160e-04, 2.4531e-02], dtype=torch.float64,
       grad_fn=<DivBackward0>)


In [35]:
print(a);a.grad.zero_()
tr = ENet(a,1.5).sum()
tr.backward()
print(a.grad)
a.grad.zero_()
print(ENet_grad(a,1.5))

tensor([4.9568e-05, 1.6942e-01, 1.8412e-04, 3.1989e-02, 1.0935e-05, 1.6087e-05,
        8.6427e-06, 3.4080e-04, 1.2265e-02], dtype=torch.float64,
       requires_grad=True)
tensor([0.5000, 0.5847, 0.5001, 0.5160, 0.5000, 0.5000, 0.5000, 0.5002, 0.5061],
       dtype=torch.float64)
tensor([0.5000, 0.5847, 0.5001, 0.5160, 0.5000, 0.5000, 0.5000, 0.5002, 0.5061],
       dtype=torch.float64, grad_fn=<DivBackward0>)


This notebook is now ready for experimentation, extension, and probabaly correction - particularly with regrads to interpretation of 
$$ \left[ \frac{\partial P(a)}{\partial \left| a_{j} \right|} \right]_{a=\hat{a}(\nu)}  \hspace{1in} (25) $$
The domain of $ \hat{R}(a) $, $\{a_{j}\}_{1}^{n}$, is not $\mathbb{R}^{n}_{+}$ but $\mathbb{R}^{n}$. Thus, $\{a_{j}\}_{1}^{n}$ can freely approach and cross $0$ as part of the evolution of $ \hat{R}(a) $. $P(a)$ should be considered $ f:\mathbb{R}^{n} \to \mathbb{R}_{+} $. Equation (25) is expressed as partial with respect to $\left| a_{j} \right|$ which seems to imply
$$ \left[ \frac{\partial P(a)}{ \partial a_{j}} 
\frac{ \partial a_{j}}{\partial \left| a_{j} \right|}
\right]_{a=\hat{a}(\nu)}  \hspace{1in} (25') $$

Need to think about this more. $
\frac{ \partial a_{j}}{\partial \left| a_{j} \right|} $ ...
Oriented space? Otherwise has little bearing. Re-read Tao's paper - He gave a very clear exposition.